In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import glob
from skimage import io
from skimage.transform import resize, resize_local_mean
import random
import matplotlib.pyplot as plt
import pandas as pd
import subprocess as sp
import rasterio as rio
import matplotlib.patches as patches
import cv2
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [ ]:
im_name = 'im_ab_0000372736-0000186368_68'
im_path = './data/reservoirs_10band/img_dir/val/{}.tif'.format(im_name)
mask_path = './data/sentinel/{}_mask.png'.format(im_name)
output_path = './data/preds/reservoirs_10band_manet_v11/ann_dir/val/{}results.png'.format(im_name)

In [ ]:

im_ar = io.imread(im_path)
mask_ar = io.imread(mask_path)
output_ar = io.imread(output_path)

In [ ]:
plt.imshow(mask_ar, interpolation='none', cmap='Blues',vmax=300)
plt.axis('off')
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch0/mask_example.png', bbox_inches='tight',transparent=True, pad_inches=0)

In [ ]:

plt.imshow(output_ar[:,-500:], interpolation='none', cmap='Blues', vmax=300)
plt.axis('off')
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch0/output_example.png', bbox_inches='tight',transparent=True, pad_inches=0)

In [ ]:

cmap_lookup={
    0: 'Blues',
    1: 'Greens',
    2: 'Reds',
    3: 'plasma',
    4: 'cividis',
    5: 'cividis',
    6: 'viridis',
    7: 'viridis',
    8: 'viridis',
    9: 'viridis',
}

In [ ]:
for b in range(im_ar.shape[2]):
    
    if b<3:
        plt.imshow(im_ar[:,:,b], interpolation='none', cmap=cmap_lookup[b], vmax=100)
    else:
        plt.imshow(im_ar[:,:,b], interpolation='none', cmap=cmap_lookup[b])
    plt.axis('off')
    plt.savefig('/home/ksolvik/research/reservoirs/figs/ch0/im_b{}_example.png'.format(b), bbox_inches='tight',transparent=True, pad_inches=0)

# Single pretty fig

In [ ]:
# RGB

im_name = 'im_0000093184-0000139776_19'
im_path = './data/reservoirs_10band/img_dir/val/{}.tif'.format(im_name)
mask_path = './data/sentinel/{}_mask.png'.format(im_name)
output_path = './data/preds/reservoirs_10band_manet_v11/ann_dir/val/{}results.png'.format(im_name)

im_ar = io.imread(im_path)
mask_ar = io.imread(mask_path)
output_ar = io.imread(output_path)[:, -500:]

In [ ]:
mask_ar.shape

In [ ]:
output_ar_rgb = mask_ar.copy()
output_ar_rgb[:,:,0] = output_ar
output_ar_rgb[:,:,1:3] = 0
output_ar_rgb[:,:,3] = output_ar

In [ ]:

fig, axs = plt.subplots(1,2, figsize=(10,5))
mask_ar[:,:,1:3] = 0
# output_ar[:,:,1:3] = 0
axs[0].axis('off')
axs[1].axis('off')
axs[0].imshow((im_ar[70:-70,70:-70,[2,1,0]]/255)*1.5)
axs[0].imshow(output_ar_rgb, cmap='hsv', alpha=0.9) # interpolation='none'
axs[0].set_xlim(0,500)
axs[0].set_ylim(0,500)
rect = patches.Rectangle((215, 230), 100, 100, linewidth=1, edgecolor='r', facecolor='none')
axs[0].add_patch(rect)
axs[1].imshow((im_ar[300:400,285:385, [2,1,0]]/255)*1.5)
# axs[1].imshow(mask_ar[230:330,215:315], alpha=0.3, cmap='hsv')
axs[1].set_xlim(0,100)
axs[1].set_ylim(0,100)
axs[1].axis('on')
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].yaxis.set_label_position("right")
axs[1].set_ylabel('1 km', labelpad=0)
axs[1].set_xlabel('1 km', labelpad=0)
axs[1].set_xticks([0,100])
axs[1].set_yticks([0,100])
axs[1].yaxis.tick_right()
axs[1].spines['left'].set_visible(False)
axs[1].spines['top'].set_visible(False)
axs[1].tick_params(labelbottom=False, labelright=False,
                      axis='both', length=7, direction='inout')    
axs[1].spines['right'].set_position(('outward', 6))  # Move left spine outward by 10 points
axs[1].spines['bottom'].set_position(('outward', 6)) 

axs[0].axis('on')
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].yaxis.set_label_position("right")
axs[0].set_ylabel('5 km', labelpad=0)
axs[0].set_xlabel('5 km', labelpad=0)
axs[0].set_xticks([0,500])
axs[0].set_yticks([0,500])
axs[0].yaxis.tick_right()
axs[0].spines['left'].set_visible(False)
axs[0].spines['top'].set_visible(False)
axs[0].tick_params(labelbottom=False, labelright=False,
                      axis='both', length=7, direction='inout')    
axs[0].spines['right'].set_position(('outward', 6))  # Move left spine outward by 10 points
axs[0].spines['bottom'].set_position(('outward', 6)) 
fig.tight_layout()

In [ ]:

contours, _ = cv2.findContours(output_ar,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
polygons = []

for obj in contours:
    coords = []
        
    for point in obj:
        coords.append(int(point[0][0]))
        coords.append(int(point[0][1]))

    polygons.append(coords)


In [ ]:


fig, ax = plt.subplots(1,1, figsize=(3.75,3.75))
mask_ar[:,:,1:3] = 0
# output_ar[:,:,1:3] = 0
ax.axis('off')
ax.imshow((im_ar[70:-70,70:-70,[2,1,0]]/255)*1.5)
ax.imshow(output_ar_rgb, cmap='hsv', alpha=0.9) # interpolation='none'
ax.set_xlim(0,500)
ax.set_ylim(0,500)

ax.axis('on')
ax.set_xticks([])
ax.set_yticks([])
ax.yaxis.set_label_position("right")
ax.set_ylabel('5 km', labelpad=0)
ax.set_xlabel('5 km', labelpad=0)
ax.set_xticks([0,500])
ax.set_yticks([0,500])
ax.yaxis.tick_right()
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(labelbottom=False, labelright=False,
                      axis='both', length=7, direction='inout')    
ax.spines['right'].set_position(('outward', 6))  # Move left spine outward by 10 points
ax.spines['bottom'].set_position(('outward', 6)) 


# Set inset axis
inset_ax =ax.inset_axes(
   [0.6, 0.6, 0.4, 0.4],  # [x, y, width, height] w.r.t. axes
    xlim=[253, 276], ylim=[263, 286], # sets viewport &amp; tells relation to main axes
    xticklabels=[], yticklabels=[],

)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
ax.indicate_inset_zoom(inset_ax, edgecolor="blue")
inset_ax.imshow((im_ar[70:,70:,[2,1,0]]/255)*1.8)
poly = patches.Polygon(contours[0][:,0], fill=False, color='red', antialiased=True, alpha=0.5, linestyle='--')
inset_ax.add_patch(poly)
for spine in inset_ax.spines.values():
    spine.set_edgecolor('blue')

fig.tight_layout()
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch0/res_example.jpg'.format(b), bbox_inches='tight',transparent=True, dpi=300)

# Landsat comparison

In [ ]:

im_name = 'im_0000186368-0000139776_70'
im_path = './data/landsat8_v9_sr/img_dir/val/{}.tif'.format(im_name)
sen_im_path = './data/reservoirs_10band/img_dir/val/{}.tif'.format(im_name)
mask_path = './data/sentinel/{}_mask.png'.format(im_name)
output_path = './data/preds/reservoirs_10band_manet_v11/ann_dir/val/{}results.png'.format(im_name)

im_ar = io.imread(im_path)
sen_im_ar = io.imread(sen_im_path)
# im_ar_30m = resize_local_mean(resize_local_mean(im_ar, (213,213)), (640,640))*255
im_ar_30m = resize_local_mean(im_ar, (213,213))*255
mask_ar = io.imread(mask_path)
output_ar = io.imread(output_path)[:, -500:]

In [ ]:
output_ar_rgb = mask_ar.copy()
output_ar_rgb[:,:,0] = output_ar
output_ar_rgb[:,:,1:3] = 0

output_ar_rgb[:,:,3] = output_ar

In [ ]:


contours, _ = cv2.findContours(output_ar,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
polygons = []

for obj in contours:
    coords = []
        
    for point in obj:
        coords.append(int(point[0][0]))
        coords.append(int(point[0][1]))

    polygons.append(coords)


In [ ]:



mask_ar[:,:,1:3] = 0
fig, axs = plt.subplots(1,3, figsize=(10,4))
for ax in axs:
    ax.set_xlim(0,500)
    ax.set_ylim(0,500)
    ax.axis('on')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.yaxis.set_label_position("right")
    ax.set_ylabel('5 km', labelpad=0)
    ax.set_xlabel('5 km', labelpad=0)
    ax.set_xticks([0,500])
    ax.set_yticks([0,500])
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(labelbottom=False, labelright=False,
                        axis='both', length=7, direction='inout')    
    ax.spines['right'].set_position(('outward', 6))  # Move left spine outward by 10 points
    ax.spines['bottom'].set_position(('outward', 6)) 

# First, sentinel
ax = axs[0]
ax.imshow((sen_im_ar[70:-70,70:-70,[2,1,0]]/255)*1.5)
# ax.imshow(output_ar_rgb, cmap='hsv', alpha=0.9) # interpolation='none'


# # Set inset axis
inset_ax =ax.inset_axes(
   [0.0, 0.4, 0.6, 0.6],  # [x, y, width, height] w.r.t. axes
    xlim=[390, 440], ylim=[115, 165], # sets viewport &amp; tells relation to main axes
    xticklabels=[], yticklabels=[],

)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
mark_inset(ax, inset_ax, loc1=1, loc2=3, fc="none", ec="blue", lw=0.7)
inset_ax.imshow((sen_im_ar[70:,70:,[2,1,0]]/255)*1.5)
for spine in inset_ax.spines.values():
    spine.set_edgecolor('blue')

# Finall, Landsat 8 30m
ax = axs[1]
ax.set_xlim(0,500/3)
ax.set_ylim(0,500/3)
ax.set_xticks([0,500/3])
ax.set_yticks([0,500/3])

ax.imshow((im_ar_30m[23:-23,23:-23,[3,2,1]]/255)*1.5)
# ax.imshow(output_ar_rgb, cmap='hsv', alpha=0.9) # interpolation='none'

# # Set inset axis
inset_ax =ax.inset_axes(
   [0.0, 0.4, 0.6, 0.6],  # [x, y, width, height] w.r.t. axes
    xlim=[390/3, 440/3], ylim=[115/3, 165/3], # sets viewport &amp; tells relation to main axes
    xticklabels=[], yticklabels=[],

)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
mark_inset(ax, inset_ax, loc1=1, loc2=3, fc="none", ec="blue", lw=0.7)
inset_ax.imshow((im_ar_30m[23:,23:,[3,2,1]]/255)*1.5)
for spine in inset_ax.spines.values():
    spine.set_edgecolor('blue')

# Then, Landsat 8 10m
ax = axs[2]
ax.imshow((im_ar[70:-70,70:-70,[3,2,1]]/255)*1.5)
# ax.imshow(output_ar_rgb, cmap='hsv', alpha=0.9) # interpolation='none'


# # Set inset axis
inset_ax =ax.inset_axes(
   [0.0, 0.4, 0.6, 0.6],  # [x, y, width, height] w.r.t. axes
    xlim=[390, 440], ylim=[115, 165], # sets viewport &amp; tells relation to main axes
    xticklabels=[], yticklabels=[],

)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
mark_inset(ax, inset_ax, loc1=1, loc2=3, fc="none", ec="blue", lw=0.7)
# ax.indicate_inset_zoom(inset_ax, edgecolor="blue")
inset_ax.imshow((im_ar[70:,70:,[3,2,1]]/255)*1.5)
for spine in inset_ax.spines.values():
    spine.set_edgecolor('blue')



axs[0].set_title('Sentinel-2, 10m')
axs[1].set_title('Landsat 8, 30m')
axs[2].set_title('Landsat 8, resampled to 10m')

fig.tight_layout()

# Tiles

In [ ]:

np_rng = np.random.default_rng(seed=1)

In [ ]:
all_imgs = glob.glob('./data/reservoirs_10band_v11/img_dir/train/*.tif')

In [ ]:
random_four =np_rng.choice(all_imgs, 4) 

In [ ]:
def add_headers(
    fig,
    *,
    row_headers=None,
    col_headers=None,
    row_pad=1,
    col_pad=5,
    rotate_row_headers=True,
    **text_kwargs
):
    # Based on https://stackoverflow.com/a/25814386

    axes = fig.get_axes()

    for ax in axes:
        sbs = ax.get_subplotspec()

        # Putting headers on cols
        if (col_headers is not None) and sbs.is_first_row():
            ax.annotate(
                col_headers[sbs.colspan.start],
                xy=(0.5, 1),
                xytext=(0, col_pad),
                xycoords="axes fraction",
                textcoords="offset points",
                ha="center",
                va="baseline",
                **text_kwargs,
            )

        # Putting headers on rows
        if (row_headers is not None) and sbs.is_first_col():
            ax.annotate(
                row_headers[sbs.rowspan.start],
                xy=(0, 0.5),
                xytext=(-ax.yaxis.labelpad - row_pad, 0),
                xycoords=ax.yaxis.label,
                textcoords="offset points",
                ha="right",
                va="center",
                rotation=rotate_row_headers * 90,
                **text_kwargs,
            )

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(10,5))
for j, ax_pair in enumerate(axs.T):
    ax_pair[0].axis('off')
    ax_pair[1].axis('off')
    im_ar = io.imread(random_four[j])
    ax_pair[0].imshow(im_ar[70:-70,70:-70,[2,1,0]])
    ax_pair[1].imshow(im_ar[70:-70,70:-70,[4]], cmap='grey')
    ax_pair[0].set_xlim(0,500)
    ax_pair[0].set_ylim(0,500)
    ax_pair[1].set_xlim(0,500)
    ax_pair[1].set_ylim(0,500)
    fig.tight_layout()
axs[0,0].axis('on')
axs[0,0].set_xticks([])
axs[0,0].set_yticks([])
axs[0,0].set_ylabel('Sentinel-2 RGB', size=14)
axs[1,0].axis('on')
axs[1,0].set_xticks([])
axs[1,0].set_yticks([])
axs[1,0].set_ylabel('Sentinel-1 VV', size=14)
axs[1,-1].axis('on')
axs[1,-1].set_xticks([])
axs[1,-1].set_yticks([])
axs[1,-1].yaxis.set_label_position("right")
axs[1,-1].set_ylabel('5 km', labelpad=0)
axs[1,-1].set_xlabel('5 km', labelpad=0)
axs[1,-1].set_xticks([0,500])
axs[1,-1].set_yticks([0,500])
axs[1,-1].yaxis.tick_right()
axs[1,-1].spines['left'].set_visible(False)
axs[1,-1].spines['top'].set_visible(False)
axs[1,-1].tick_params(labelbottom=False, labelright=False,
                      axis='both', length=7, direction='inout')    
axs[1,-1].spines['right'].set_position(('outward', 6))  # Move left spine outward by 10 points
axs[1,-1].spines['bottom'].set_position(('outward', 6)) 